In [3]:
from bs4 import BeautifulSoup
import requests

In [1]:
FROM = 'tehran'
TO = 'lahijan'
DATE = '1401/12/24'

In [2]:
url = f'https://safar724.com/bus/{FROM}-{TO}?date={DATE}'

In [5]:
soup = BeautifulSoup(requests.get(url).content, 'html.parser')
soup

<!DOCTYPE html>

<!--[if IE 9]> <html lang="fa" class="ie9"> <![endif]-->
<!--[if !IE]><!-->
<html dir="ltr" lang="fa">
<!--<![endif]-->
<head>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1.0" name="viewport"/>
<meta content="RQMBA" name="fontiran.com:license">
<title data-bind="htmlDocumentTitle: ('بلیط اتوبوس ' + selectedOrigin.PersianName() +' '+ selectedDestination.PersianName() + ' : قیمت ، ساعات حرکت ، خرید آنلاین - سفر 724')"></title>
<meta data-bind="metaContent: ('سفر ۷۲۴ سامانه رزرو آنلاین بلیط اتوبوس VIP و معمولی ' + selectedOrigin.PersianName() +' '+ selectedDestination.PersianName() + ' از تمامی شرکت های مسافربری و ترمینال ها')" name="description"/>
<!-- Twitter Card data -->
<meta content="summary" name="twitter:card"/>
<meta content="@safar_724" name="twitter:site"/>
<meta data-bind="metaContent: ('بلیط اتوبوس ' + selectedOrigin.PersianName() +' '+ selectedDestination.PersianName() + ' : قیمت ، ساعات حرکت ، خرید آنلاین - سفر 724')" name="twit